### Imports

#### Import Packages

In [1]:
import polars as pl
import altair as alt
import numpy as np
from great_tables import GT
import gc

# alt.JupyterChart.enable_offline()
# alt.renderers.enable("jupyter", offline=True)

#### Import Panel Data

In [2]:
grocery_lf = pl.scan_csv(source="data/panel-datasets/edible_grocery.csv",
                         has_header=True,
                         separator=",",
                         schema={'panel_id': pl.Int32,
                                 'trans_id': pl.Int32,
                                 'week': pl.Int16,
                                 'sku_id': pl.Categorical,
                                 'units': pl.Int16,
                                 'price': pl.Float32,
                                 'brand': pl.Categorical})
grocery_lf.head().collect()

panel_id,trans_id,week,sku_id,units,price,brand
i32,i32,i16,cat,i16,f32,cat
3102011,1569,6,"""19""",1,2.79,"""Alpha"""
3102012,4301,15,"""15""",1,3.19,"""Alpha"""
3102012,4301,15,"""15""",1,3.19,"""Alpha"""
3102012,4301,15,"""38""",1,3.49,"""Bravo"""
3102012,4301,15,"""44""",1,3.49,"""Bravo"""


In [3]:
sku_lf = pl.scan_csv(source="data/panel-datasets/sku_weight.csv",
                         has_header=True,
                         separator=",",
                         schema={'sku_id': pl.Categorical,
                                 'weight': pl.Int16})
sku_lf.head().collect()

sku_id,weight
cat,i16
"""1""",400
"""2""",400
"""3""",400
"""4""",250
"""5""",1000


In [4]:
kiwi_lf = pl.scan_csv(source="data/panel-datasets/kiwibubbles_trans.csv",
                      has_header=True,
                      separator=",",
                      schema={'ID': pl.Int16,
                              'Market': pl.Categorical,
                              'Week': pl.Int16,
                              'Day': pl.Int16,
                              'Units': pl.Int16})
kiwi_lf.head().collect()

ID,Market,Week,Day,Units
i16,cat,i16,i16,i16
10001,"""1""",19,3,1
10002,"""1""",12,5,1
10003,"""1""",37,7,1
10004,"""1""",30,6,1
10004,"""1""",47,3,1


#### Reusable Functions

In [5]:
def weekly_plot(dataframe, y, color=None, title="", y_axis_label="", pct=False, legend=False):
    
    # Configure the color encoding only if color is provided
    if color is not None:
        color_encoding = alt.Color(
            f'{color}:N',  # N = a discrete unordered category
            legend=alt.Legend(title=color) if legend else None  # Add legend conditionally
        )
    else:
        color_encoding = alt.Color()  # No color encoding    
    
    chart = alt.Chart(dataframe).mark_line(strokeWidth=1).encode(
        x = alt.X(
            'week',
            axis=alt.Axis(
                values=np.arange(0, 104 + 1, 13), # Explicitly specify quarter-end weeks
                labelExpr="datum.value", # Show only these labels
                title='Week'
            )
        ),
        y = alt.Y(
            f'{y}:Q', # Q = a continuous real-valued quantity
            title=y_axis_label,
            axis=alt.Axis(format="$,.0f") if not pct else alt.Axis(format=",.0%")
        ),
        color = color_encoding
    ).properties(
        width=650,
        height=250,
        title=title
    ).configure_view(
        stroke=None
    ).configure_axisY(
        # grid=False # turn off y-axis grid if required
    )

    return alt.JupyterChart(chart)

### Preliminaries

#### Weekly Sales Pattern

In [6]:
# Grocery Sales LazyFrame (Query Plan): Category Weekly 'spend' - units x price = spend
grocery_spend_category = (
    grocery_lf
    .select(
        ['week', 'units', 'price']
    )
    .with_columns(
        ((pl.col('units') * pl.col('price')).cast(pl.Float64)).alias('spend')
    )
    .group_by('week')
    .agg(
        pl.col("spend").sum().cast(pl.Float64).alias('Weekly Spend')
    )
    .sort('week')
)

In [7]:
# Grocery Sales LazyFrame (Query Plan): Weekly 'spend' by Brand - units x price = spend
grocery_spend_brand = (
    grocery_lf
    .select(
        ['week', 'units', 'price', 'brand']
    )
    .with_columns(
         ((pl.col('units') * pl.col('price')).cast(pl.Float64)).alias('spend')
    )
    .group_by('week', 'brand')
    .agg(
        pl.col("spend").sum().cast(pl.Float64).alias('Weekly Spend')
    )
    .sort('week')
)

In [8]:
# Weekly Sales Pivot Table - Polars DataFrame
# For visualizing and inspecting only
weekly_sales_pivot = grocery_spend_brand.collect().pivot(
    on="brand",
    index="week",
    values="Weekly Spend",
    sort_columns=True,
).with_columns(
    pl.sum_horizontal(pl.exclude('week')).cast(pl.Float64).alias("Total") # Row total
).sort("week")

weekly_sales_pivot

week,Alpha,Bravo,Charlie,Delta,Other,Total
i16,f64,f64,f64,f64,f64,f64
1,331.459999,247.780003,51.79,17.64,18.75,667.420002
2,567.249997,398.640007,45.43,34.19,23.23,1068.740004
3,497.969999,639.980008,51.11,39.39,14.55,1243.000006
4,1389.960004,472.500004,51.87,85.04,39.779999,2039.150007
5,358.239999,252.540003,40.21,69.54,45.93,766.460002
…,…,…,…,…,…,…
100,692.480006,420.479996,69.600001,54.590001,47.97,1285.120004
101,265.770002,264.659999,64.030001,35.05,26.88,656.390003
102,379.610005,456.78999,43.5,20.4,43.61,943.909995


In [9]:
plotdata = grocery_spend_category.collect()

weekly_plot(dataframe=plotdata, 
            y='Weekly Spend', 
            title='Category - Weekly Revenue', 
            y_axis_label='Spend ($)',
            pct=False,
            legend=False)

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300, 'stroke': None}, 'axis…

In [10]:
plotdata = grocery_spend_brand.filter(
    pl.col('brand') == 'Alpha'
).collect()

weekly_plot(dataframe=plotdata, 
            y='Weekly Spend', 
            color='brand',
            title='Alpha - Weekly Revenue', 
            y_axis_label='Spend ($)',
            pct=False,
            legend=False)

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300, 'stroke': None}, 'axis…

In [11]:
plotdata = grocery_spend_brand.filter(
    pl.col('brand') == 'Bravo'
).collect()

weekly_plot(dataframe=plotdata, 
            y='Weekly Spend', 
            color='brand',
            title='Bravo - Weekly Revenue', 
            y_axis_label='Spend ($)',
            pct=False,
            legend=False)

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300, 'stroke': None}, 'axis…

#### Weekly (Volume) Market Share

In [12]:
# Grocery Volume LazyFrame: Weekly 'volume' - units x weight = volume
grocery_volume = (
    grocery_lf
    .join(
        other=sku_lf,
        left_on="sku_id",
        right_on="sku_id"
    )
    .select(
        ['week', 'units', 'brand', 'weight']
    )
    .with_columns(
        # volume column that is the product of weight of each SKU and the units of SKU sold
        (((pl.col('units') * pl.col('weight'))/1000).cast(pl.Float64)).alias('volume') # # weight from grams to kilograms
    )
    .group_by('week', 'brand')
    .agg(
        pl.col("volume").sum().cast(pl.Float64).alias('Weekly Volume')
    )
    .sort('week')
)

In [13]:
# Weekly (volume) market share
pct_volume =(
    grocery_volume
    .collect()
    .pivot(
        on='brand',
        index='week',
        values='Weekly Volume',
        sort_columns=True
    )
    .with_columns(
        (pl.col("*").exclude("week")) / pl.sum_horizontal(pl.exclude('week')).cast(pl.Float64)
    )
)

pct_volume

week,Alpha,Bravo,Charlie,Delta,Other
i16,f64,f64,f64,f64,f64
1,0.481077,0.355011,0.11194,0.02532,0.026652
2,0.524953,0.371525,0.059391,0.029696,0.014435
3,0.389338,0.508812,0.060676,0.032505,0.008668
4,0.677032,0.230511,0.036686,0.044984,0.010787
5,0.428735,0.324306,0.07574,0.119348,0.051871
…,…,…,…,…,…
100,0.545707,0.31711,0.080381,0.039087,0.017715
101,0.36891,0.40552,0.140805,0.059138,0.025627
102,0.342581,0.543494,0.071147,0.019689,0.023089


In [14]:
pct_volume_plot = (
    grocery_volume
    .group_by('week')
    .agg(
        pl.col('Weekly Volume').sum().alias('Total Weekly Volume')
    )
    .join(
        other=grocery_volume,
        on='week',
    )
    .filter(
        (pl.col('brand') == 'Alpha') |
        (pl.col("brand") == 'Bravo')
    )
    .with_columns(
        # compute brand wise % of total volume sale
        (pl.col('Weekly Volume') / pl.col('Total Weekly Volume')).alias('pct_volume')
    )
).collect()

weekly_plot(dataframe=pct_volume_plot, 
            y='pct_volume', 
            color='brand', 
            title="", 
            y_axis_label="", 
            pct=True, 
            legend=True)

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300, 'stroke': None}, 'axis…

In [15]:
corr_matrix = (
    pct_volume.select(
        pl.col('*').exclude('week')  # Exclude 'week' column
    )
    .corr()  # Compute the correlation matrix
    .with_columns(
        pl.Series(pct_volume.columns[1:]).alias("index")
    )
)

(
    GT(corr_matrix, rowname_col='index')
    .tab_header(title="Correlation Matrix of Weekly (Volume) Market Share")
    .fmt_number(columns=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'])
    .data_color(
        domain=[-1, 1],
        palette=["rebeccapurple", "white", 'orange']
    )
)

GT(_tbl_data=shape: (5, 6)
┌───────────┬───────────┬───────────┬───────────┬───────────┬─────────┐
│ Alpha     ┆ Bravo     ┆ Charlie   ┆ Delta     ┆ Other     ┆ index   │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---     │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆ f64       ┆ str     │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═════════╡
│ 1.0       ┆ -0.853061 ┆ -0.34158  ┆ -0.109693 ┆ -0.148193 ┆ Alpha   │
│ -0.853061 ┆ 1.0       ┆ -0.114179 ┆ -0.231641 ┆ -0.104983 ┆ Bravo   │
│ -0.34158  ┆ -0.114179 ┆ 1.0       ┆ 0.131988  ┆ 0.243434  ┆ Charlie │
│ -0.109693 ┆ -0.231641 ┆ 0.131988  ┆ 1.0       ┆ 0.308354  ┆ Delta   │
│ -0.148193 ┆ -0.104983 ┆ 0.243434  ┆ 0.308354  ┆ 1.0       ┆ Other   │
└───────────┴───────────┴───────────┴───────────┴───────────┴─────────┘, _body=<great_tables._gt_data.Body object at 0x10cf85be0>, _boxhead=Boxhead([ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10cf85940>, _spanners=Spanners([]), _heading=Heading(title='Correlation Matrix of Weekly (Volume) Market Share', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ffa500')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7c51a8')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[2]), grpname=None, colname='Alpha', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#cbb9dc')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[3]), grpname=None, colname='Alpha', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#eee9f4')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[4]), grpname=None, colname='Alpha', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#e8e1f0')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7c51a8')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=Non

#### Annual Sales

In [16]:
# Annual sales by brand
annual_sales = (
    grocery_spend_brand
    .select(
        ['week', 'Weekly Spend', 'brand']
    )
    .with_columns(
        (pl.col("week") / 52).ceil().cast(pl.Int32).alias('year')
    )
    .group_by(['year', 'brand'])
    .agg(
        pl.col("Weekly Spend").sum().cast(pl.Float64).alias('Yearly Sales')
    )
    .sort('year')
)

In [17]:
annual_sales_pivot = annual_sales.collect().pivot(
    on='brand',
    index='year',
    sort_columns=True
).with_columns(
    pl.sum_horizontal(pl.all()).alias("Total") # add totals colum
)

(
    GT(annual_sales_pivot, rowname_col="year")
    .tab_header(title="Annual Sales ($) by Brand")
    .tab_stubhead(label="Year")
    .fmt_currency()
    .data_color(
        columns=annual_sales_pivot.columns[:-1],
        domain=[100, 36_000],
        palette=["white", "rebeccapurple"]
    )    
)

GT(_tbl_data=shape: (2, 7)
┌──────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┬──────────────┐
│ year ┆ Alpha        ┆ Bravo        ┆ Charlie     ┆ Delta       ┆ Other       ┆ Total        │
│ ---  ┆ ---          ┆ ---          ┆ ---         ┆ ---         ┆ ---         ┆ ---          │
│ i32  ┆ f64          ┆ f64          ┆ f64         ┆ f64         ┆ f64         ┆ f64          │
╞══════╪══════════════╪══════════════╪═════════════╪═════════════╪═════════════╪══════════════╡
│ 1    ┆ 33570.940269 ┆ 28603.350202 ┆ 5120.870036 ┆ 3271.509991 ┆ 1535.229996 ┆ 72102.900494 │
│ 2    ┆ 35250.75023  ┆ 26926.869781 ┆ 3922.68004  ┆ 2820.810008 ┆ 1739.819996 ┆ 70662.930055 │
└──────┴──────────────┴──────────────┴─────────────┴─────────────┴─────────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x10d04c770>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.stub: 2>, column_label='year', column_align='right', column_width=None), ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='Total', type=<ColInfoTypeEnum.default: 1>, column_label='Total', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10cf96710>, _spanners=Spanners([]), _heading=Heading(title='Annual Sales ($) by Brand', subtitle=None, preheader=None), _stubhead='Year', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='year', rows=[0]), grpname=None, colname='year', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[1]), grpname=None, colname='year', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7041a0')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#69379b')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#865dae')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8d67b3')]), StyleInfo(locname=LocBody(columns='Charlie', rows=[0]), grpname=None, colname='Charlie', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, s

In [18]:
annual_change = annual_sales_pivot.with_columns(
    pl.col('*').exclude('year').pct_change()
).filter(
    pl.col('year') == 2
).unpivot(
    index="year",  # Keep 'year' as a fixed identifier
    value_name='% Change',  # New column for melted column names (e.g., brand names)
    variable_name='brand'
)

(
    GT(annual_change, rowname_col='brand')
    .tab_header(title='% Change in Annual Sales')
    .tab_stubhead(label="Brands")
    .fmt_percent()
    .data_color(
        domain=[-0.3, 0.3],
        palette=['orange', 'white', 'rebeccapurple']
    ) 
    .cols_hide('year')   
    .cols_label(brand='Y-o-Y % Change')
)

GT(_tbl_data=shape: (6, 3)
┌──────┬─────────┬───────────┐
│ year ┆ brand   ┆ % Change  │
│ ---  ┆ ---     ┆ ---       │
│ i32  ┆ str     ┆ f64       │
╞══════╪═════════╪═══════════╡
│ 2    ┆ Alpha   ┆ 0.050038  │
│ 2    ┆ Bravo   ┆ -0.058611 │
│ 2    ┆ Charlie ┆ -0.233982 │
│ 2    ┆ Delta   ┆ -0.137765 │
│ 2    ┆ Other   ┆ 0.133263  │
│ 2    ┆ Total   ┆ -0.019971 │
└──────┴─────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x10d041480>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.hidden: 4>, column_label='year', column_align='right', column_width=None), ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='Y-o-Y % Change', column_align='left', column_width=None), ColInfo(var='% Change', type=<ColInfoTypeEnum.default: 1>, column_label='% Change', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10d054550>, _spanners=Spanners([]), _heading=Heading(title='% Change in Annual Sales', subtitle=None, preheader=None), _stubhead='Brands', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='year', rows=[0]), grpname=None, colname='year', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[1]), grpname=None, colname='year', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[2]), grpname=None, colname='year', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[3]), grpname=None, colname='year', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[4]), grpname=None, colname='year', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[5]), grpname=None, colname='year', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', r

In [22]:
chart = alt.Chart(annual_sales.collect()).mark_bar().encode(
    x=alt.X("brand:N", 
            axis=alt.Axis(labelAngle=0)),
    xOffset="year:N",
    y=alt.Y("Yearly Sales:Q",
            axis=alt.Axis(format="$,.0f")),
    color=alt.Color("year:N"),
).properties(
    width=650,
    height=250,
    title='Year 1 & Year 2 Revenues'
).show()

alt.JupyterChart(chart)

alt.Chart(...)

JupyterChart()

In [23]:
base = alt.Chart(annual_change).encode(
    x=alt.X("brand:N", 
            axis=alt.Axis(labelAngle=0)
    ),
    y=alt.Y("% Change:Q",
            axis=alt.Axis(format=".0%")
    ),
    color=alt.condition(
        alt.datum["% Change"] > 0, # fixed this 
        alt.value("green"),  # The positive color
        alt.value("red")  # The negative color
    ),
    text=alt.Text(
        '% Change',
        format=(".0%")
    )
        
).properties(
    width=650,
    height=250,
    title='Y-o-Y % Change in Revenue'
)

chart = base.mark_bar() + base.mark_text(align='center', 
                                 baseline=alt.expr(alt.expr.if_(alt.datum['% Change'] >= 0, 'bottom', 'top')),
                                 dx=0)

alt.JupyterChart(chart)

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300}}, 'layer': [{'mark': {…

#### Annual Market Share

In [25]:
annual_sales.collect()

year,brand,Yearly Sales
i32,cat,f64
1,"""Bravo""",28603.350202
1,"""Other""",1535.229996
1,"""Delta""",3271.509991
1,"""Charlie""",5120.870036
1,"""Alpha""",33570.940269
2,"""Delta""",2820.810008
2,"""Bravo""",26926.869781
2,"""Other""",1739.819996
2,"""Alpha""",35250.75023


In [26]:
annual_sales_pivot

year,Alpha,Bravo,Charlie,Delta,Other,Total
i32,f64,f64,f64,f64,f64,f64
1,33570.940269,28603.350202,5120.870036,3271.509991,1535.229996,72102.900494
2,35250.75023,26926.869781,3922.68004,2820.810008,1739.819996,70662.930055
